In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# Simple Impact-T Run

In [2]:
from impact import Impact

In [3]:
ipath = '../templates/lcls_injector/'
ifile = '../templates/lcls_injector/ImpactT.in'

In [4]:
I = Impact(ifile, verbose=True)

/Users/chrisonian/Code/GitHub/lume-impact/templates/lcls_injector
Configured for tempdir: /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/temp_impactT_0mbup2ut


In [5]:
print(I)

================ Impact-T Summary ================
1 bunchtotal charge: 249.99999999999997 pC
Distribution type: read
Cathode start at z = 0 m
   emission time: 1.4e-11 s
   image charges neglected after z = 0.02 m
Tracking 300004 particles
Processor domain: 1 x 1 = 1 CPUs
Computational domain: 0.015 m x 0.015 m x 45.0 m
Space charge grid: 64 x 64 x 128
Maximum time steps: 1000000
Random Seed: 6
Reference Frequency: 2856000000.0 Hz
Initial reference time: 0.0 s
Simulation starting from the beginning

Impact-T configured in /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/temp_impactT_0mbup2ut


In [6]:
# Change some things
I.input['header']['Np'] = 3004
I.input['header']['Nx'] = 16
I.input['header']['Ny'] = 16
I.input['header']['Nz'] = 16
I.input['header']['Dt'] = 5e-13

In [9]:
# Change stop location
I.ele['stop_1']['s'] = 0.01

In [10]:
I.timeout = 1000
I.run()

partcl.data already exits, will not overwrite.
Loaded fort 30 : Fourth root of the fourth moments of the beam distribution
Loaded fort 25 : RMS Y information
Loaded fort 24 : RMS X information
Loaded fort 26 : RMS Z information
Loaded fort 28 : Load balance and loss diagnostics
Loaded fort 29 : Cube root of third moments of the beam distribution
Loaded fort 18 : Time and energy
Loaded fort 27 : Max amplitude information
Loaded fort 70 : Slice information of the final distribution
Loaded fort 60 : Slice information of the initial distribution
Loaded fort 40 : initial particle distribution at t = 0
Loaded fort 50 : final particle distribution projected to the centroid location of the bunch


In [11]:
I.particles

{'initial_particles': array([( 4.3611631e-04,  1.2615826e-03,  3.2937175e-04, -5.7131252e-04, -2.7340319e-06, 0.001978),
        ( 4.0614893e-04,  1.0219854e-03,  3.8438809e-04,  7.9314477e-04, -1.7480634e-06, 0.001978),
        (-4.6228725e-06, -9.2105950e-05, -1.0653409e-04, -3.6416778e-05, -2.1070141e-06, 0.001978),
        ...,
        ( 3.1444710e-04, -1.0118680e-03,  4.0362795e-05,  1.0135250e-03, -3.3099046e-06, 0.001978),
        ( 2.3358952e-04, -8.0413991e-05, -4.8289365e-05, -1.2245925e-03, -3.2073538e-06, 0.001978),
        (-4.7831343e-04, -1.8353668e-03,  1.9170274e-04, -4.4979501e-04, -1.5912101e-06, 0.001978)],
       dtype=[('x', '<f8'), ('GBx', '<f8'), ('y', '<f8'), ('GBy', '<f8'), ('z', '<f8'), ('GBz', '<f8')]),
 'final_particles': array([( 0.00016224,  0.01061123, -1.01499853e-03, -0.0610029 , 0.01030399, 2.83196904),
        ( 0.00015552,  0.00866931,  3.76046702e-04,  0.02136602, 0.01073721, 2.86220828),
        ( 0.00118522,  0.08358915, -3.51321900e-05, -0.00149

In [12]:
I.output['stats'].keys()

dict_keys(['t', 'z', 'x_moment4', 'GBx_moment4', 'y_moment4', 'GBy_moment4', 'z_moment4', 'GBz_moment4', 'y_centroid', 'y_rms', 'GBy_centroid', 'GBy_rms', 'y_twiss', 'y_normemit', 'x_centroid', 'x_rms', 'GBx_centroid', 'GBx_rms', 'x_twiss', 'x_normemit', 'z_centroid', 'z_rms', 'GBz_centroid', 'GBz_rms', 'z_twiss', 'z_normemit', 'numparticles_min', 'numparticles_max', 'numparticles', 'x_moment3', 'GBx_moment3', 'y_moment3', 'GBy_moment3', 'z_moment3', 'GBz_moment3', 'gamma', 'E_kinetic', 'beta', 'r_max', 'deltaGamma_rms', 'x_max', 'GBx_max', 'y_max', 'GBy_max', 'z_max', 'GBz_max'])

In [13]:
I.output['slice_info']

{'final_particle_slices': {'slice_z': array([-0.00195693, -0.00169601, -0.00143509, -0.00117416, -0.00091324,
         -0.00065231, -0.00039139, -0.00013046,  0.00013046,  0.00039139,
          0.00065231,  0.00091324,  0.00117416,  0.00143509,  0.00169601,
          0.00195693]),
  'particles_per_cell': array([  64.08197, 1250.373  , 3556.697  , 1949.189  ,  654.0843 ,
          908.352  ,  863.0922 ,  568.0007 ,  430.6301 ,  554.2596 ,
          864.019  ,  895.93   ,  929.0265 ,  914.6713 ,  853.9784 ,
          366.6151 ]),
  'current': array([  6.127469, 119.5597  , 340.0886  , 186.38    ,  62.54304 ,
          86.85592 ,  82.52821 ,  54.31179 ,  41.17652 ,  52.99787 ,
          82.61683 ,  85.66813 ,  88.8328  ,  87.46017 ,  81.65675 ,
          35.05545 ]),
  'x_emittance': array([1.169690e-06, 2.106731e-06, 1.417134e-06, 2.163778e-06,
         3.502870e-06, 2.471005e-06, 1.884877e-06, 2.103940e-06,
         2.222641e-06, 1.644615e-06, 1.221919e-06, 1.176429e-06,
         1.0659

# Plot

In [14]:
from bokeh.plotting import output_notebook
from bokeh.plotting import show
output_notebook(hide_banner=True)

from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Range1d
from impact.parsers import load_many_fort
from impact.parsers import UNITS

from impact.plotting import layout_plot


ds = ColumnDataSource(I.output['stats'])

In [15]:
I.ele['quadrupole_1']

{'description': 'QA01',
 'original': '0.204    10 20 1   4.752856    1.8524   0.108 0.016 0.0 0.0 0.0 0.0 0.0 /  !QA01',
 'L': 0.204,
 'itype': 1,
 'type': 'quadrupole',
 'zedge': 4.752856,
 'b1_gradient': 1.8524,
 'L_effective': 0.108,
 'radius': 0.016,
 'x_offset': 0.0,
 'y_offset': 0.0,
 'x_rotation': 0.0,
 'y_rotation': 0.0,
 'z_rotation': 0.0,
 's': 4.956856,
 'name': 'quadrupole_1'}

In [18]:
# Change somethone
I.ele['quadrupole_1']['b1_gradient'] = 20
layout = layout_plot(I.input['lattice'], plot_height=500)
show(layout)

In [19]:
def plot1(y_axis, x_axis='z', source=ds):
    yunit = UNITS[y_axis]
    ylabel = y_axis+' ('+yunit+')'
    
    xunit = UNITS[x_axis]
    xlabel = x_axis+' ('+xunit+')'
    p = figure(plot_width=600, plot_height=200, x_axis_label=xlabel, y_axis_label=ylabel)
    p.line( x_axis, y_axis, source=source, color='red')
    return p
show(plot1('x_rms'))